本篇的目的是介绍蒙特卡洛模拟法的步骤，主要是如何使用蒙特卡洛法获取衍生品，尤其是期权的价格

考虑在第一篇中讲过的几何布朗运动：
$$
dS = bSdt+\sigma S dz \tag{4.1}
$$
若设$Y = lnS$,根据伊藤引理，有：
$$
dY = \frac{\partial Y}{\partial S} dS + \frac{1}{2}\frac{\partial^2 Y}{\partial S^2} (dS)^2 \tag{4.2}
$$

计算偏导并将偏导和(4.1)带入，可得：
$$
dY = (b-\frac{1}{2}\sigma^2)dt + \sigma dz \tag{4.3}
$$

故而积分可得：
$$
Y_t - Y_0 = (b-\frac{1}{2}\sigma^2)t + \sigma (z_t-z_0) \tag{4.4}
$$

因为$z_0 =0,Y_0=lnS_0$,所以有：
$$
Y_t =lnS_0 +  (b-\frac{1}{2}\sigma^2)t + \sigma z_t \tag{4.5}
$$
其$z_t = \epsilon\sqrt{t}(\epsilon \sim N(0,1))$为标准正态分布的随机变量，所以最后解得：
$$
S_t = S_0 e^{(b-\frac{1}{2}\sigma^2)t + \sigma \epsilon\sqrt{t}} \tag{4.6}
$$


根据(4.6)即可模拟出一段时间内路径内的衍生品价格。

（1） 模拟欧式期权

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import scipy.stats as stats
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def geo_brownian(steps,paths,T,S0,b,sigma):
    #模拟路劲生成
    dt = T/steps #时间间隔dt
    S_path = np.zeros((steps+1,paths)) #初始化路径矩阵,，用来准备储存模拟情况
    S_path[0] = S0 #起点设置
    for step in range(1,steps+1):
        rn = np.random.standard_normal(paths) #创造随机数
        S_path[step] = S_path[step-1] * np.exp((b-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*rn)
    return S_path
    

In [3]:
def MC_Euro_Option(steps,paths,T,S0,X,sigma,b,r):
    #模拟欧式期权价格
    S_path = geo_brownian(steps,paths,T,S0,b,sigma) #模拟路径
    value = np.exp(-r*T)*np.maximum(S_path[-1]-X,0).mean()
    return value

In [4]:
C = MC_Euro_Option(steps=250,paths=5000000,T=1,S0=100,X=99,sigma=0.2,b=0.03,r=0.03)
print(C)

9.926707877251987


这一结果和BSM算出来的解析解十分接近

考虑亚式期权

亚式期权的到期回报取决于路劲上资产价格的均值（算术平均或几何平均），这很难得到解析解，因此需要使用蒙特卡洛模拟

In [9]:
def asian_option(steps,paths,T,S0,X,sigma,r,b):
    S_path = geo_brownian(steps,paths,T,S0,b,sigma) #生成路径
    S_mean = S_path.mean(axis=0) #计算每一条路劲上的均价
    value = np.exp(-r*T)*np.maximum(S_mean-X,0).mean() #计算到期回报的均值
    return value


In [10]:
asian_option(steps=250,paths=500000,T=1,S0=100,X=99,sigma=0.2,r=0.03,b=0.03)

5.8046783320159685

In [11]:
S_path = geo_brownian(steps=250,paths=500000,T=1,S0=100,b=0.03,sigma=0.2) 

In [17]:
S_path

array([[100.        , 100.        , 100.        , ..., 100.        ,
        100.        , 100.        ],
       [100.84928316,  98.60083621,  98.85683424, ..., 101.28876654,
        100.39831892, 101.53210003],
       [ 98.57406305, 100.99797115, 101.53789308, ..., 100.78473922,
        100.42835543, 101.5660064 ],
       ...,
       [102.74318092,  97.95230661, 116.27053534, ...,  85.24073779,
        119.1982379 , 102.47979816],
       [100.95117222,  98.73564136, 117.55611439, ...,  85.47398257,
        118.47164904, 100.99012852],
       [100.30526422,  98.53446219, 119.03426157, ...,  86.16858852,
        116.63597706, 100.34851427]])